# Image Processing SS 20 - Assignment - 08

### Deadline is 17.6.2020 at 11:55am

Please solve the assignments together with a partner.
I will run every notebook. Make sure the code runs through. Select `Kernel` -> `Restart & Run All` to test it.
Please strip the output from the cells, either select `Cell` -> `All Output` -> `Clear` or use the `nb_strip_output.py` script / git hook.

In [ ]:
# display the plots inside the notebook
%matplotlib inline

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pylab
from skimage.data import chelsea
from skimage.color import rgb2gray
from numpy.fft import fft2 as numpy_fft2, ifft2 as numpy_ifft2

pylab.rcParams['figure.figsize'] = (12, 12)   # This makes the plot bigger

# Exercise 1 - 2D DCT - 4 Points
Transform a picture with 2D DCT (discrete cosine transform), which you must implement manually using DCT-II.
For the 2D-DCT, perform two 1D-DCT-transformations. Therefore, implement a method which returns a normalized DCT-II
matrix. Plot a normalized 16x16 DCT-II-matrix (with colors reflecting the values in the matrix).

In [ ]:
def dct2d(img):
    """
    Returns the 2d discrete cosine transformation
    """
    # your code here
    return numpy_fft2(img)

def inv_dct2d(x):
    """
    Returns the 2d inverse discrete cosine transformation
    """
    # your code here
    return numpy_ifft2(x)

def chess_board(n=8, field_size=32):
    board = np.zeros((n*field_size, n*field_size))
    s = field_size
    for i in range(n):
        for j in range(n):
            if (i + j) % 2 == 0:
                board[i*s:(i+1)*s, j*s:(j+1)*s] = 1
    return board



for pic in [chess_board(), rgb2gray(chelsea())/255]:
    plt.subplot(121)
    plt.imshow(np.real(dct2d(pic)), cmap='gray')
    plt.subplot(122)
    plt.imshow(np.imag(dct2d(pic)), cmap='gray')
    plt.show()

# Exercise 2 - 1 Point: (Manual calculation, no code needed.)

Given is a signal [5,10,12]. Now you have to mirror the signal as in the lectures (resulting in a signal of length 5) and perform a DFT. Please write down all coefficients in the frequency spectrum. What do you see (one sentence)?

# Exercise 3 - High and Low Pass filter with 2D DCT - 5 Points

Implement the `high_pass` and `low_pass` functions which work on the frequency spectrum of a 2D-DCT
transformed image. Apply the filters to both the chessboard and cat image and transform the
result back into image space.

Show the spectrum before and after filtering as well as the resulting images after backtransformation
into image space.

In [ ]:
def high_pass(img_ft, n):
    """Removes the low frequencies"""
    # your code here
    ft = img_ft.copy()
    return ft

def low_pass(img_ft, n):
    """Removes the high frequencies"""
    # your code here
    ft = img_ft.copy()
    return ft
    
def inv_dct_and_plot(img_ft):
    plt.imshow(np.real(inv_dct2d(img_ft)), cmap='gray')
    plt.show()

In [ ]:
img = rgb2gray(chelsea() / 255)
# remove the low frequencies
chelsea_high_freq = high_pass(dct2d(img), 12)
inv_dct_and_plot(chelsea_high_freq)

In [ ]:
# remove the high frequencies
chelsea_low_freq = low_pass(dct2d(img), 12)
inv_dct_and_plot(chelsea_low_freq)